In [1]:
#!pip install glob2
!wget https://storage.googleapis.com/download.tensorflow.org/models/tflite/gpu/multi_person_mobilenet_v1_075_float.tflite

--2019-03-28 23:01:04--  https://storage.googleapis.com/download.tensorflow.org/models/tflite/gpu/multi_person_mobilenet_v1_075_float.tflite
Resolving storage.googleapis.com... 172.217.168.240, 2a00:1450:400e:80d::2010
Connecting to storage.googleapis.com|172.217.168.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5048148 (4.8M) [application/octet-stream]
Saving to: ‘multi_person_mobilenet_v1_075_float.tflite.1’

multi_person_mobile 100%[===================>]   4.81M  --.-KB/s    in 0.08s   

2019-03-28 23:01:04 (58.5 MB/s) - ‘multi_person_mobilenet_v1_075_float.tflite.1’ saved [5048148/5048148]



In [2]:
import glob2
import os
from random import randint
from IPython.core.display import Image, display
import numpy as np
import numpy as np
import tensorflow as tf
from skimage import transform,io
#import PIL
#from PIL import Image

In [3]:
training_path = "/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/"
glob_search_path = os.path.join(training_path, "**/*.jpg")
jpg_paths = glob2.glob(glob_search_path)
print("jpg_paths", len(jpg_paths))

jpg_paths 97


In [4]:
index = randint(0,len(jpg_paths)-1)
img_filename = jpg_paths[index]
print("File %s: %s" % (index, img_filename))
#display(Image(filename=img_filename, width=257, height=353))

File 36: /whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_104_95409.57449405201.jpg


In [5]:
image = io.imread(img_filename)
print(image.shape)
image_rot = transform.rotate(image,90.0,resize=True)
print(image_rot.shape)
small_input = transform.resize(image_rot,(353,257), mode='symmetric', preserve_range=True)
# reshape to (1,784)

reshape_img = small_input.reshape(1, 353,257, 3)
print(reshape_img.shape)

(1080, 1920, 3)
(1920, 1080, 3)
(1, 353, 257, 3)


/anaconda/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [6]:
# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path="./multi_person_mobilenet_v1_075_float.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)
print(interpreter.get_input_details())

input_data = np.array(reshape_img, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)
#print(output_data[0][0][0])

[  1 353 257   3]
[{'name': 'sub_2', 'index': 97, 'shape': array([  1, 353, 257,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
(1, 23, 17, 17)


In [7]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [8]:

import PIL.Image
#from cStringIO import StringIO
import io
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = io.StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [9]:
#PIL.Image.fromarray(image)

In [10]:

#imshow(small_input)
#imshow(image_rot)
#imshow(image)